In [1]:
import re
from tqdm import tqdm
from utils import line_reader

import spacy
from spacy.matcher import Matcher
from spacy.tokens import Doc, Token

from nltk.corpus import sentiwordnet as swn

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
HEADLINES_FILE = '../../../tasks/02-structural-linguistics/examiner-headlines.txt'
n_lines = sum(1 for _ in line_reader(HEADLINES_FILE))

In [4]:
nlp = spacy.load('en_core_web_md')

### Завдання

1. Напишіть програму, яка аналізує заголовок за першими трьома ознаками (у спрощеній формі)
    * Чи є в заголовку іменовані стуності?
    * Чи є заголовок позитивно чи негативно забарвлений?
    * Чи є в заголовку прикметники та прислівники вищого і найвищого ступенів порівняння?
2. Проженіть вашу програму на корпусі заголовків з The Examiner. Для кожної з трьох ознак, визначте відсоток заголовків у корпусі, які її мають.
3. Збережіть програму та пораховану статистику в директорії з вашим іменем.

Додаткова інформація:
* Типи сутностей, які впливають на "вірусність" заголовка, виберіть самостійно.
* Для визначення емоційного забарвлення, використайте SentiWordNet. Наприклад, можна перевірити, що середнє значення позитивності/негативності слова у заголовку перевищує 0.5. Для визначення середнього значення можна брати до п'яти перших значень слова з такою частиною мови. Будьте креативними та експериментуйте.

In [5]:
pos_to_santi = {'NOUN': 'n', 'VERB': 'v', 'ADJ' : 'a', 'ADV' : 'r'}
N_SENTI_WORDS = 5

def get_senti_tag(token):
    try:
        return pos_to_santi[token.pos_]
    except KeyError:
        return None
    
def get_senti_score(token):
    if token._.senti_tag:
        senti_words = list(swn.senti_synsets(token.lower_, pos=token._.senti_tag))[:N_SENTI_WORDS]
        senti_score = sum(max(sw.pos_score(), sw.neg_score()) for sw in senti_words)
        senti_score /= N_SENTI_WORDS
    else:
        senti_score = 0
    return senti_score

Token.set_extension('senti_tag', getter=get_senti_tag, force=True)
Token.set_extension('senti_score', getter=get_senti_score, force=True)

In [6]:
VIRAL_ENTS = ('PERSON', 'NORP', 'EVENT', 'MONEY', 'ORG')

matcher = Matcher(nlp.vocab)
matcher.add('SUPERLATIVE', None, [{'TAG': 'JJR'}], [{'TAG': 'JJS'}], 
 [{'TAG': 'RBS'}, {'TAG': 'JJ'}], [{'TAG': 'RBR'}, {'TAG': 'JJ'}],
 [{'TAG': 'RBS'}, {'TAG': 'RB'}], [{'TAG': 'RBR'}, {'TAG': 'RB'}])


def is_viral_ent(doc):
    for ent in doc.ents:
        if ent.label_ in VIRAL_ENTS:
            return True
    return False


def is_emotional(doc):
    for t in doc:
        if t._.senti_score > 0.5:
            return True
    return False


def is_superlative(doc):
    return True if matcher(doc) else False


Doc.set_extension('is_viral_ent', getter=is_viral_ent, force=True)
Doc.set_extension('is_emotional', getter=is_emotional, force=True)
Doc.set_extension('is_superlative', getter=is_superlative, force=True)


In [7]:
headlines = line_reader(HEADLINES_FILE)

stats = {
    'n_viral_ents': 0,
    'n_emotional' : 0,
    'n_superlative': 0
    }

for doc in tqdm(nlp.pipe(headlines), total=n_lines):
    stats['n_viral_ents'] += int(doc._.is_viral_ent)
    stats['n_emotional'] += int(doc._.is_emotional)
    stats['n_superlative'] += int(doc._.is_superlative)

100%|██████████| 5000/5000 [00:27<00:00, 180.35it/s]


In [9]:
for stat, count in stats.items():
    print('{:>15} {:<4.1f}%'.format(stat, 100 * count/n_lines))

   n_viral_ents 59.3%
    n_emotional 3.5 %
  n_superlative 4.3 %
